# Daily MME Meta Analysis
Adapting a method recently developed by FDA to analyze a [related opioid methods question](https://www.fda.gov/media/141914/download), we used meta analytic techniques to test the impact of the four definitions in the real-world. The general set up is to compare opioid use in FL vs. CA across the 4 definitions of daily MME. We previously observed that Florida had higher unadjusted levels of opioid use, presumably an interaction with an older population and the enactment of clinical pain management legislation. We took two approaches, 1) comparing the proportion of "high dose" users among opioid recipients, and 2) comparing average daily MME between the states, stratified by medicines used for acute versus chronic pain.

## Comparing "High Dose" patients in CA and FL

Input dataset from table of high dose patients (>90 daily MME) among adult outpatient opioid recipients identified using the PDMP of each state. `boundary=0` designates if it is greater than 90 MME and `boundary=1` designates greater than or equal to 90 daily MME. This dataset is not actually used in the analysis but is the underlying raw data for subsequent steps.

In [1]:
clear all
qui: input str2 state definition highdose population boundary
"CA" 1 87078 2430870 0
"CA" 2 140822 2430870 0
"CA" 3 86407 2430870 0
"CA" 4 249471 2430870 0
"CA" 1 106240 2430870 1
"CA" 2 155254 2430870 1
"CA" 3 87407 2430870 1
"CA" 4 285807 2430870 1
"FL" 1 87295 1485591 0
"FL" 2 136995 1485591 0
"FL" 3 97346 1485591 0
"FL" 4 211429 1485591 0
"FL" 1 113998 1485591 1
"FL" 2 157794 1485591 1
"FL" 3 98541 1485591 1
"FL" 4 261335 1485591 1
end

* Create numeric indicator for state
gen staten=1 if state=="FL"
    replace staten=0 if state=="CA"




. gen staten=1 if state=="FL"
(8 missing values generated)

.     replace staten=0 if state=="CA"
(8 real changes made)


---
Generate Rate Ratios with California `staten=0` reference group.

In [2]:
di "===== Proportion of high dose patients FL vs CA greater than 90 daily MME ====="
* definition 1
    csi  87295 87078  1485591 2430870
* definition 2
    csi 136995 140822 1485591 2430870
* definition 3
    csi 97346 86407 1485591 2430870
* definition 4
    csi 211429 249471 1485591 2430870


===== Proportion of high dose patients FL vs CA greater than 90 daily MME =====


                 |   Exposed   Unexposed  |      Total
-----------------+------------------------+------------
           Cases |     87295       87078  |     174373
        Noncases |   1485591     2430870  |    3916461
-----------------+------------------------+------------
           Total |   1572886     2517948  |    4090834
                 |                        |
            Risk |  .0554999    .0345829  |   .0426253
                 |                        |
                 |      Point estimate    |    [95% Conf. Interval]
                 |------------------------+------------------------
 Risk difference |          .020917       |    .0204939      .02134 
      Risk ratio |         1.604835       |    1.590181    1.619625 
 Attr. frac. ex. |          .376883       |    .3711406    .3825731 
 Attr. frac. pop |          .188676       |
                 +-------------------------------------

---
Scrape "Risk ratio" into new input dataset. Create log-transformed variables to meet normal distribution assumption of meta analytic statistics.

In [3]:
clear all
qui: input definition irr ll ul str31 label
1  1.604835      1.590181    1.619625     "D1. Sum of days supply" 
2   1.541862     1.530841    1.552962    "D2. Accounting for overlap days"    
3   1.791581    1.775632    1.807674   "D3. Defined observation window"    
4  1.33859     1.331294    1.345926     "D4. Maximum daily dose"   
end

gen lnirr=ln(irr)
gen lnll=ln(ll)
gen lnul=ln(ul)

qui: meta set lnirr lnll lnul, studylabel(label) 




. gen lnirr=ln(irr)

. gen lnll=ln(ll)

. gen lnul=ln(ul)

. qui: meta set lnirr lnll lnul, studylabel(label)


---
Run meta analysis command using fixed effects REML model. Since there is no sampling variation, fixed effects is the preferred *a priori* specification.

In [4]:
meta summarize, fixed eform


  Effect-size label:  Effect Size
        Effect size:  lnirr
          Std. Err.:  _meta_se
        Study label:  label

Meta-analysis summary                                Number of studies =      4
Fixed-effects model                                  Heterogeneity:
Method: Inverse-variance                                       I2 (%) =   99.91
                                                                   H2 = 1148.14

-------------------------------------------------------------------------------
                       Study |        exp(ES)    [95% Conf. Interval]  % Weight
-----------------------------+-------------------------------------------------
      D1. Sum of days supply |          1.605       1.590       1.620     15.37
D2. Accounting for overlap~s |          1.542       1.531       1.553     25.14
D3. Defined observation wi~w |          1.792       1.776       1.808     16.18
      D4. Maximum daily dose |          1.339       1.331       1.346     43.31
--------

For the sake of completeness, random effects models are also run, using the Sidik-Jonkman `random(sj)` estimator because tau is expected to be large [Veroniki et al.](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4950030/), with  DerSimonian–Laird `random(dl)` as well separately for comparison.

In [5]:
meta summarize, random(sj) eform


  Effect-size label:  Effect Size
        Effect size:  lnirr
          Std. Err.:  _meta_se
        Study label:  label

Meta-analysis summary                                Number of studies =      4
Random-effects model                                 Heterogeneity:
Method: Sidik-Jonkman                                            tau2 =  0.0145
                                                               I2 (%) =   99.90
                                                                   H2 = 1004.19

-------------------------------------------------------------------------------
                       Study |        exp(ES)    [95% Conf. Interval]  % Weight
-----------------------------+-------------------------------------------------
      D1. Sum of days supply |          1.605       1.590       1.620     24.99
D2. Accounting for overlap~s |          1.542       1.531       1.553     25.00
D3. Defined observation wi~w |          1.792       1.776       1.808     24.99
      D4

In [6]:
meta summarize, random(dl) eform


  Effect-size label:  Effect Size
        Effect size:  lnirr
          Std. Err.:  _meta_se
        Study label:  label

Meta-analysis summary                                Number of studies =      4
Random-effects model                                 Heterogeneity:
Method: DerSimonian-Laird                                        tau2 =  0.0166
                                                               I2 (%) =   99.91
                                                                   H2 = 1148.14

-------------------------------------------------------------------------------
                       Study |        exp(ES)    [95% Conf. Interval]  % Weight
-----------------------------+-------------------------------------------------
      D1. Sum of days supply |          1.605       1.590       1.620     24.99
D2. Accounting for overlap~s |          1.542       1.531       1.553     25.00
D3. Defined observation wi~w |          1.792       1.776       1.808     24.99
      D4

Results are similar, but SJ is preferred based on simulations in Veroniki et al. The fixed effects model over emphasizes precision (e.g., confuses it for more information) in D4 due to the higher number of high dose patients. Since there is no sampling variation 

## Interpretation
The proportion of "high dose" patients was consitently higher in Florida across all variants. However, the magnitude of the difference varied greatly: 79% (95% CI: 78%, 81%) for Definition 3 (defined observation window); 60% (95% CI: 59%, 62%) for Definition 1 (sum of days supply); 54% (95% CI: 53%, 55%) for Definition 2 (accounting for overlap days); and 34% (95% CI: 33%, 35%) for Definition 4 (maximum daily dose). Metrics confirmed very high heterogenity between the definitions, with I2 greater than 99% and H2 of 1148, supported by tests of hetereogenity chi2 of 3444 on 3 degrees of freedom (p<0.0001), and overall effect z=219, with 1 degree of freedom and p<0.0001.

---
# Meta Analysis of Means by Type of Opioid

In this meta analysis we examine the impact of definitional variation on acute vs. chronic pain patients, measured by opioid formulation type. We stratified the sample into three sub-groups: 1) patients receiving on only immediate-release or short-acting opioids labeled for acute pain (hereafter immediate-release; 2) patients receiving only extended-release or long-acting opioids generally labeled for chronic pain (hereafter extended-release); and 3) patients receiving both immediate-release and extended-release opioids contemporaneously within the 3 month observation period (e.g., chronic pain patients receiving opioids for breakthrough pain or during taper).

Input data from analysts with mean (and SE) of daily MME, with stratum-specific population, by formulation and state.

In [7]:
clear all
qui: input str2 state definition avg se population formulation str80 label
"CA" 1 30.3156249 0.1477 2273028 1 "D1. California Sum of days supply"
"CA" 2 31.5819604 0.1479 2273028 1 "D2. California Accounting for overlap days"
"CA" 3 10.3398905 0.0282 2273028 1 "D3. California Defined observation window"
"CA" 4 39.6430507 0.1860 2273028 1 "D4. California Maximum daily dose"
"FL" 1 34.0531498 0.0246 1338828 1 "D1. Florida Sum of days supply"
"FL" 2 35.0964146 0.0261 1338828 1 "D2. Florida Accounting for overlap days"
"FL" 3 12.5794512 0.0219 1338828 1 "D3. Florida Defined observation window"
"FL" 4 44.7478467 0.0418 1338828 1 "D4. Florida Maximum daily dose"

"CA" 1 90.2232825 0.5002 40038 2 "D1. California Sum of days supply"
"CA" 2 103.7573329 0.6715 40038 2 "D2. California Accounting for overlap days"
"CA" 3 72.753132 0.5228 40038 2 "D3. California Defined observation window"
"CA" 4 153.6802569 1.0256 40038 2 "D4. California Maximum daily dose"
"FL" 1 86.9071545 0.5450 26039 2 "D1. Florida Sum of days supply"
"FL" 2 96.9302372 0.6372 26039 2 "D2. Florida Accounting for overlap days"
"FL" 3 66.8367252 0.5028 26039 2 "D3. Florida Defined observation window"
"FL" 4 143.0437107 0.9884 26039 2 "D4. Florida Maximum daily dose"

"CA" 1 74.1906194 0.1876 117804 3 "D1. California Sum of days supply"
"CA" 2 143.9839494 0.4413 117804 3 "D2. California Accounting for overlap days"
"CA" 3 122.7372442 0.4328 117804 3 "D3. California Defined observation window"
"CA" 4 250.7462218 0.8219 117804 3 "D4. California Maximum daily dose"
"FL" 1 82.95423 0.1703 120724 3 "D1. Florida Sum of days supply"
"FL" 2 160.1525421 0.3788 120724 3 "D2. Florida Accounting for overlap days"
"FL" 3 133.0969773 0.3625 120724 3 "D3. Florida Defined observation window"
"FL" 4 267.949697 0.6850 120724 3 "D4. Florida Maximum daily dose"
end

gen staten=1 if state=="FL"
    replace staten=0 if state=="CA"




. gen staten=1 if state=="FL"
(12 missing values generated)

.     replace staten=0 if state=="CA"
(12 real changes made)


---
## Immediate-release only
Continuing with the approach in the previous meta analysis, we compared the average daily MME between Florida and California. We used negative binomial (NB2) regression to estimate the relative difference (ratio). NB2 was used instead of Poisson due to overdispersion. `formulation==1` designates the IR-only category. No linear transformation needed in this meta analysis because average MME is continuous.

In [8]:
di "--Definition 1--"
glm avg staten [fw=population] if formulation==1 & definition==1, link(log) family(nb) eform nohead 
di "--Definition 2--"
glm avg staten [fw=population] if formulation==1 & definition==2, link(log) family(nb) eform nohead
di "--Definition 3--"
glm avg staten [fw=population] if formulation==1 & definition==3, link(log) family(nb) eform nohead
di "--Definition 4--"
glm avg staten [fw=population] if formulation==1 & definition==4, link(log) family(nb) eform nohead


--Definition 1--

note: avg has noninteger values

Iteration 0:   log likelihood =  -16146496  
Iteration 1:   log likelihood =  -16146496  

------------------------------------------------------------------------------
             |                 OIM
         avg |        IRR   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
      staten |   1.123287   .0012424   105.11   0.000     1.120855    1.125725
       _cons |   30.31562   .0204367  5060.82   0.000     30.27559    30.35571
------------------------------------------------------------------------------
Note: _cons estimates baseline incidence rate.

--Definition 2--

note: avg has noninteger values

Iteration 0:   log likelihood =  -16277871  
Iteration 1:   log likelihood =  -16277871  

------------------------------------------------------------------------------
             |                 OIM
         avg |        IRR   Std. Err.      z

Effect measure and standard errors scraped into a new dataset for meta analysis, and put into a new frame

In [9]:
frame create ir
frame change ir

qui: input definition rr se str80 label
1 1.123287 .0012424 "D1. Sum of days supply"
2 1.11128 .0012285 "D2. Accounting for overlap days"
3 1.216594 .0013811 "D3. Defined observation window"
4 1.128769 .001244 "D4. Maximum daily dose"
end

qui: meta set rr se , studylabel(label) 





. qui: meta set rr se , studylabel(label)


---
## Extended-release only

In [10]:
frame change default

di "--Definition 1--"
glm avg staten [fw=population] if formulation==2 & definition==1, link(log) family(nb) eform nohead 
di "--Definition 2--"
glm avg staten [fw=population] if formulation==2 & definition==2, link(log) family(nb) eform nohead
di "--Definition 3--"
glm avg staten [fw=population] if formulation==2 & definition==3, link(log) family(nb) eform nohead
di "--Definition 4--"
glm avg staten [fw=population] if formulation==2 & definition==4, link(log) family(nb) eform nohead



--Definition 1--

note: avg has noninteger values

Iteration 0:   log likelihood = -362969.87  
Iteration 1:   log likelihood = -362969.87  

------------------------------------------------------------------------------
             |                 OIM
         avg |        IRR   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
      staten |   .9632454   .0077119    -4.68   0.000     .9482483    .9784797
       _cons |   90.22328   .4533942   895.93   0.000     89.33901    91.11631
------------------------------------------------------------------------------
Note: _cons estimates baseline incidence rate.

--Definition 2--

note: avg has noninteger values

Iteration 0:   log likelihood = -371363.94  
Iteration 1:   log likelihood = -371363.94  

------------------------------------------------------------------------------
             |                 OIM
         avg |        IRR   Std. Err.      

In [11]:
frame create er
frame change er

input definition rr se str80 label
1 .9632454 .0077119 "D1. Sum of days supply"
2 .9342013 .0074746 "D2. Accounting for overlap days"
3 .9186782 .0073665 "D3. Defined observation window"
4 .9307879 .0074353 "D4. Maximum daily dose"
end

qui: meta set rr se , studylabel(label) 





     definit~n         rr         se                                            
>                                  label



---
## Both Extended-release and Immediate-release

In [12]:
frame change default

di "--Definition 1--"
glm avg staten [fw=population] if formulation==3 & definition==1, link(log) family(nb) eform nohead 
di "--Definition 2--"
glm avg staten [fw=population] if formulation==3 & definition==2, link(log) family(nb) eform nohead
di "--Definition 3--"
glm avg staten [fw=population] if formulation==3 & definition==3, link(log) family(nb) eform nohead
di "--Definition 4--"
glm avg staten [fw=population] if formulation==3 & definition==4, link(log) family(nb) eform nohead



--Definition 1--

note: avg has noninteger values

Iteration 0:   log likelihood = -1280775.8  
Iteration 1:   log likelihood = -1280775.8  

------------------------------------------------------------------------------
             |                 OIM
         avg |        IRR   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
      staten |   1.118123   .0046083    27.09   0.000     1.109127    1.127192
       _cons |   74.19061   .2176087  1468.29   0.000     73.76533    74.61835
------------------------------------------------------------------------------
Note: _cons estimates baseline incidence rate.

--Definition 2--

note: avg has noninteger values

Iteration 0:   log likelihood = -1437573.4  
Iteration 1:   log likelihood = -1437573.3  

------------------------------------------------------------------------------
             |                 OIM
         avg |        IRR   Std. Err.      

In [13]:
frame create erir
frame change erir

input definition rr se str80 label
1 1.118123 .0046083 "D1. Sum of days supply"
2 1.112294 .0045703 "D2. Accounting for overlap days"
3 1.084406 .0044584  "D3. Defined observation window"
4 1.068609 .0043848 "D4. Maximum daily dose"
end

qui: meta set rr se , studylabel(label) 





     definit~n         rr         se                                            
>                                  label



---
## Meta Regression
Meta regression is being used to assess heterogeneity, not to derive a summary effect measure. Since there is no sampling variation between studies, fixed effect models are used, but random effects models were also run to check for major divergent results (none were found, data not shown but can be run with the commented-out code below).

### IR only

In [14]:
frame ir {
    meta summarize, fixed
    *meta summarize, random(sj)
}



  Effect-size label:  Effect Size
        Effect size:  rr
          Std. Err.:  se
        Study label:  label

Meta-analysis summary                                Number of studies =      4
Fixed-effects model                                  Heterogeneity:
Method: Inverse-variance                                       I2 (%) =   99.92
                                                                   H2 = 1293.65

-------------------------------------------------------------------------------
                       Study |    Effect Size    [95% Conf. Interval]  % Weight
-----------------------------+-------------------------------------------------
      D1. Sum of days supply |          1.123       1.121       1.126     26.11
D2. Accounting for overlap~s |          1.111       1.109       1.114     26.71
D3. Defined observation wi~w |          1.217       1.214       1.219     21.13
      D4. Maximum daily dose |          1.129       1.126       1.131     26.05
----------------

### ER only

In [15]:
frame er {
    meta summarize, fixed
    *meta summarize, random(sj)
}



  Effect-size label:  Effect Size
        Effect size:  rr
          Std. Err.:  se
        Study label:  label

Meta-analysis summary                                Number of studies =      4
Fixed-effects model                                  Heterogeneity:
Method: Inverse-variance                                       I2 (%) =   83.83
                                                                   H2 =    6.19

-------------------------------------------------------------------------------
                       Study |    Effect Size    [95% Conf. Interval]  % Weight
-----------------------------+-------------------------------------------------
      D1. Sum of days supply |          0.963       0.948       0.978     23.61
D2. Accounting for overlap~s |          0.934       0.920       0.949     25.13
D3. Defined observation wi~w |          0.919       0.904       0.933     25.87
      D4. Maximum daily dose |          0.931       0.916       0.945     25.39
----------------

### Both ER and IR

In [16]:
frame erir {
    meta summarize, fixed
    *meta summarize, random(sj)
}



  Effect-size label:  Effect Size
        Effect size:  rr
          Std. Err.:  se
        Study label:  label

Meta-analysis summary                                Number of studies =      4
Fixed-effects model                                  Heterogeneity:
Method: Inverse-variance                                       I2 (%) =   96.31
                                                                   H2 =   27.12

-------------------------------------------------------------------------------
                       Study |    Effect Size    [95% Conf. Interval]  % Weight
-----------------------------+-------------------------------------------------
      D1. Sum of days supply |          1.118       1.109       1.127     23.87
D2. Accounting for overlap~s |          1.112       1.103       1.121     24.27
D3. Defined observation wi~w |          1.084       1.076       1.093     25.50
      D4. Maximum daily dose |          1.069       1.060       1.077     26.36
----------------

## Interpretation
+ ER only group had *lower* average daily MME in Florida than California?! 
+ Heterogeneity by I^2 was high for all 3 definitions
+ Heterogeneity was lowest for ER-only group
+ For ER+IR group, the definitional variants would have resulted in us conclusing that average daily MME was higher in FL by: 6.9%, 8.4%, 11.2%, or 11.8%. Are these interesting policy or clinical distinctions?
